<a href="https://colab.research.google.com/github/MathewsJosh/Resmat/blob/main/%5BMAC015%5D_Trabalho_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elaborar um programa que dimensione para o usuário, em termos de seu diâmetro (em mm), um eixo circular que funcione acoplado a um motor que gira a uma frequência de 120 rpm e dissipa 150 cv de potência (1 cv = 736 W), considerando a solicitação à torção.

Para tal, considerar eixo maciço para os seguintes materiais:

Aço: G = 75 GPa, Tensão admissível ao cisalhamento = 50 MPa

Latão: G = 40 GPa, Tensão admissível ao cisalhamento = 48 MPa

Alumínio: G = 25 GPa, Tensão admissível ao cisalhamento = 25 MPa

Para o dimensionamento, o programa deverá levar em conta não só as tensões admissíveis de cada material, mas também uma limitação para o ângulo de torção, que não deve exceder 1º. Considerar o comprimento L do eixo como 20 vezes o diâmetro (no caso dos perfis tubulares, 20 vezes o diâmetro externo).

Fazer também o dimensionamento para perfis tubulares de aço (considerar as mesmas propriedades já citadas para o material). Deve-se ler a tabela de perfis disponíveis (para este exercício, foram escolhidos perfis Vallourec 141,3 mm), cujos dados geométricos pertinentes ao exercício foram planilhados em Excel, e verificar os tubos disponíveis em relação à tensão e ao ângulo de torção. Escolher o perfil mais econômico (de menor massa linear).

Considerar como dados de entrada a potência dissipada pelo motor e sua frequência de giro.

Fornecer como saída uma tabela indicando para o usuário:

Material

Seção escolhida (em mm)

Apresentar análise crítica sobre o "peso" da limitação da tensão x limitação do ângulo de torção no dimensionamento dos eixos.

Analisar o que acontece com o dimensionamento se:

a) A potência do motor varia

b) A frequência de giro do motor varia

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
# Cálculo do momento torsor (N.mm)

# Função que retorna diâmetro e ângulo
def retorna_dados(modulo_el, tensao_ad, limita_angulo):
  frequencia = 1 # Frequência em hertz (120 rpm)
  potencia = 150*736 # Potência convertida para W

  T = (potencia)/(frequencia*np.pi) # Torque (N.m)
  T = T*10**3 # Torque (N.mm)

  # Cálculo das propriedades geométricas. Como não sabemos o raio do eixo, vamos usar as equações deduzidas nas aulas
  raio = np.cbrt((2*T)/(np.pi*tensao_ad)) # Raio em mm
  diametro = 2*raio # Diâmetro em mm
  diametro = round(diametro) # Arredonda o valor do diâmetro para o inteiro mais superior mais próximo
    
  if limita_angulo:
    angulo = 1
    comprimento = (diametro/1000)*20 # Passa o diâmetro para metro e multiplica por 20, de acordo com o enunciado
    c = (T*comprimento)/((modulo_el*10**9)*((np.pi/2)*angulo))
    diametro = (c**0.25)*2 # Sabendo que J = (pi/2)*c^4
  else:
    momento_in = (np.pi/2)*((raio/1000)) # Passa o raio para metro e encontra o momento de inércia polar
    comprimento = (diametro*20)/1000 # Passa o diâmetro para metro e multiplica por 20, de acordo com o enunciado
    angulo = (T*comprimento)/((modulo_el*10**9)*momento_in) # Define ângulo em graus
  return diametro, np.degrees(angulo) # Retorna diâmetro em mm e ângulo em graus

# Dados dos materiais (Aço, latão e alumínio)
modulos_el = [75,40,25] # Módulos de elasticidade
tensoes_ad = [50,48,25] # Tensões admissível ao cisalhamento
lista_diametros = [] # Varíavel que será usada para armazenar os diâmetros
lista_angulos = [] # Varíavel que será usada para armazenar os ângulos

for i in range(3):
  diametro, angulo = retorna_dados(modulos_el[i], tensoes_ad[i], False)
  lista_diametros.append(diametro)
  lista_angulos.append(angulo)

# Mostra os resultados
outputDf = pd.DataFrame(
    {
        "Material":["Aço","Latão","Alumínio"], 
        "Diâmetro (mm)":lista_diametros,
        "Ângulo (graus)":lista_angulos   
    }
)
outputDf

,Material,Diâmetro (mm),Ângulo (graus)
0,Aço,153,0.683763
1,Latão,155,1.281261
2,Alumínio,193,2.053759


In [3]:
# O próximo passo seria trabalhar com uma limitação para o ângulo de torção
for i in range(3):
  if lista_angulos[i] > 1:
    diametro, angulo = retorna_dados(modulos_el[i], tensoes_ad[i], True)
  else:
    diametro, angulo = retorna_dados(modulos_el[i], tensoes_ad[i], False)